<a href="https://colab.research.google.com/github/RafaelCaballero/Julio25/blob/main/code/20log%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
### Rafa Caballero

## Regresión logística


### Índice
[Ejemplo](#ejemplo)<br>
[Clases desequilibradas](#desequilibrio)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[Undersampling](#undersampling)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[Oversampling](#oversampling)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[Smote](#smote)<br>
[Aplicación a validación cruzada](#cruzada)<br>
[Interpretación de los coeficientes](#coeficientes)<br>
[Curva ROC](#roc)<br>

<a name="ejemplo"></a>
### Ejemplo
Partimos del ejemplo de los autobuses

In [ ]:
file = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/bus.csv"
import pandas as pd

df = pd.read_csv(file)
df

In [ ]:
df.I8.hist()

Consideramos que un autobús llega tarde cuando tarda más de 580 segundos en este último trayecto.

In [ ]:
df["label"] = 1
df.loc[df.I8<580, "label"] = 0
df

Ojo porque al hacer esto tenemos que eliminar el dato I8 (¿por qué?)

In [ ]:
df2 = df.drop(columns=["I8"])

In [ ]:
df2["label"].value_counts()

In [ ]:
import matplotlib.pyplot as plt


df3 = df2.sort_values(by="label")
x=range(len(df3))
plt.scatter(x,df3.label,s=0.1)

Ejemplo de función para evaluar la clasificación

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
def evaluar(y_test,y_pred):
  k =  cohen_kappa_score(y_test,y_pred)
  print("kappa ",k)
  cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                display_labels=modelo.classes_)
  disp.plot()

  plt.show()

  print(classification_report(y_test, y_pred))
  return k,cm


In [ ]:
df2

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


# 1 dividir columnas
yColumn = "label"
XColumns = [c for c in df2.columns if c!=yColumn] # ["hora,"I0",....]
X = df[XColumns]
y = df[yColumn]

# 2 preparar train y test
test = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

# 3 método y entrenamiento
metodo = LogisticRegression(max_iter=1000)
modelo = metodo.fit(X_train,y_train)

# 4 evaluar
y_pred = modelo.predict(X_test)
evaluar(y_test,y_pred)

<a name="desequilibrio"></a>
### Clases desequilibradas
Lo que está ocurriendo es que al haber pocos 1's, el sistema no "aprende" a reconocerlos.



In [ ]:
y.value_counts()

Posibilidades:


<a name="undersampling"></a>
#### Undersampling

El undersampling o *submuestreo*  limita la cantidad del valor que más se repite, en este caso el 0. Vamos a hacerlo primero a mano para entenderlo, y luego usaremos una librería. Los dos primeros pasos son igual

In [ ]:
from sklearn.utils import shuffle

# 1 dividir columnas
yColumn = "label"
XColumns = [c for c in df2.columns if c!=yColumn]
X = df[XColumns]
y = df[yColumn]

# 2 preparar train y test
test = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

X_train, y_train


In [ ]:
train_unos = y_train[y_train==1]
train_ceros = y_train[y_train==0].sample(len(train_unos))

y_train_equilibrado = pd.concat([train_unos,train_ceros])

y_train_equilibrado = y_train_equilibrado.sample(len(y_train_equilibrado)) # para barajarlos

X_train_equilibrado = X_train.loc[y_train_equilibrado.index]


In [ ]:
y_train_equilibrado, X_train_equilibrado

Ahora tenemos las 2 clases equilibradas

In [ ]:
(y_train_equilibrado==0).sum(),(y_train_equilibrado==1).sum()

In [ ]:
# 3 método y entrenamiento
metodo = LogisticRegression(max_iter=100000)
modelo = metodo.fit(X_train_equilibrado,y_train_equilibrado)

# 4 evaluar
y_pred = modelo.predict(X_test)
evaluar(y_test,y_pred)

<a name="oversampling"></a>
#### Oversampling

El oversampling o *sobremuestreo*  añade repeticiones del valor más escaso, en este caso el 1. La idea es la misma, solo que usamos muestreo con repetición en la clase de menos elementos. Suele dar mejor resultado si hay pocos valores

In [ ]:
# 2 continuación
train_ceros = y_train[y_train==0]
train_unos = y_train[y_train==1].sample(len(train_ceros),replace=True) # con reemplazamiento

y_train_equilibrado = pd.concat([train_unos,train_ceros])
y_train_equilibrado = y_train_equilibrado.sample(len(y_train_equilibrado)) # para barajarlos
X_train_equilibrado = X_train.loc[y_train_equilibrado.index]

# 3 método y entrenamiento
metodo = LogisticRegression(max_iter=100000)
modelo = metodo.fit(X_train_equilibrado,y_train_equilibrado)

# 4 evaluar
y_pred = modelo.predict(X_test)
evaluar(y_test,y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=modelo.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=modelo.classes_)
disp.plot()

plt.show()

In [ ]:
# 3 método y entrenamiento
metodo = LogisticRegression(max_iter=100000)
modelo = metodo.fit(X_train_equilibrado,y_train_equilibrado)

# 4 evaluar
y_pred = modelo.predict(X_test)
evaluar(y_test,y_pred)

<a name="smote"></a>
#### SMOTE

Este método genera nuevos valores interpolando otros existentes para la nueva etiqueta, no duplica sin más

In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

# 2 continuación

sm = SMOTE()
X_train_equilibrado, y_train_equilibrado = sm.fit_resample(X_train, y_train)

# 3 método y entrenamiento
metodo = LogisticRegression(max_iter=100000)
modelo = metodo.fit(X_train_equilibrado,y_train_equilibrado)

# 4 evaluar
y_pred = modelo.predict(X_test)
evalua()

In [ ]:
# 3 método y entrenamiento
metodo = LogisticRegression(max_iter=100000)
modelo = metodo.fit(X_train_equilibrado,y_train_equilibrado)

# 4 evaluar
y_pred = modelo.predict(X_test)
evaluar(y_test,y_pred)

**Importante**: aplicar estas transformaciónes solo al train

Además, la biblioteca *imbalanced learn* incorpora también los métodos de undersamping y oversampling haciendo más sencilla la tarea:

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from imblearn.over_sampling import SMOTE

# 2 continuación

sm = RandomOverSampler()
X_train_equilibrado, y_train_equilibrado = sm.fit_resample(X_train, y_train)

# 3 método y entrenamiento
metodo = LogisticRegression(max_iter=100000)
modelo = metodo.fit(X_train_equilibrado,y_train_equilibrado)

# 4 evaluar
y_pred = modelo.predict(X_test)
evaluar(y_test,y_pred)

<a name="cruzada"></a>
### Aplicación a validación cruzada

Como se ve la biblioteca nos permite escribir código más simple (reduciendo la posibilidad de error), pero es que además nos permite utilizarlo con validación cruzada para obtener resultados más sencillos.

Hay que notar que la validación cruzada se complica en el caso del equilibrado porque en cada caso tenemos un test distinto y todo se hace internamente. Tenemos que pasarle a la funcion de validación un método de ML que equilibre antes de aplicar la técnica correspondiente. Esto se logra gracias a los pipelines

In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings(action='ignore')

steps = [('over', RandomOverSampler()), ('logistic', LogisticRegression(max_iter=10000))]
pipeline = Pipeline(steps=steps)
repartidor = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(pipeline, X, y, scoring='balanced_accuracy', cv=repartidor)
scores

In [ ]:
scores.mean()

In [ ]:
steps = [('over', RandomUnderSampler()), ('logistic', LogisticRegression())]
pipeline = Pipeline(steps=steps)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(pipeline, X, y, scoring='balanced_accuracy', cv=cv)
scores.mean()

Sin oversampling

In [ ]:
steps = [ ('logistic', LogisticRegression())]
pipeline = Pipeline(steps=steps)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(pipeline, X, y, scoring='balanced_accuracy', cv=cv)
scores.mean()

Ojo con usar solo accuracy sin oversampling, nos puede decir que está muy bien porque ignora la clase que se repite menos

In [ ]:
steps = [ ('logistic', LogisticRegression())]
pipeline = Pipeline(steps=steps)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv)
scores.mean()

En caso de duda lo mejor es obtener la matriz de confusión por validación cruzada, que se puede obtener a partir de
[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

steps = [ ('logistic', LogisticRegression())]
pipeline = Pipeline(steps=steps)

y_pred = cross_val_predict(pipeline, X, y, cv=10)
cm = confusion_matrix(y, y_pred, labels=modelo.classes_,normalize="true")
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=modelo.classes_)
disp.plot()

plt.show()

In [ ]:
steps = [('over', RandomOverSampler()), ('logistic', LogisticRegression())]
pipeline = Pipeline(steps=steps)
y_pred = cross_val_predict(pipeline, X, y, cv=10)
cm = confusion_matrix(y, y_pred, labels=modelo.classes_,normalize="true")
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=modelo.classes_)
disp.plot()

plt.show()

<a name="coeficientes"></a>
### Interpretación de los coeficientes

In [ ]:
modelo = LogisticRegression().fit(X,y)

In [ ]:
v1 = [8,471.0, 650, 370, 280,  1020.6,  665, 420]
v2 = [8,471.0, 650, 370, 280,  980,  640, 390]

modelo.predict_proba([v1])

In [ ]:
modelo.predict_proba([v2])

In [ ]:
pd.DataFrame(modelo.coef_,columns=X.columns)

Vemos que el que más influye es la hora, y después el I7. Es difícil interpretar el signo negativo de la hora, quizás sea mejor convertirla a one-hot encoding:

In [ ]:
X2 = X.copy()
X2["Hora"] = X.Hora.astype(str)
X2 = pd.get_dummies(X2)
X2

In [ ]:
steps = [('over', RandomUnderSampler()), ('logistic', LogisticRegression())]
pipeline = Pipeline(steps=steps)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(pipeline, X2, y, scoring='balanced_accuracy', cv=cv)
scores.mean()

In [ ]:
steps = [('over', RandomUnderSampler()), ('logistic', LogisticRegression())]
pipeline = Pipeline(steps=steps)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(pipeline, X, y, scoring='balanced_accuracy', cv=cv)
scores.mean()

In [ ]:
modelo = LogisticRegression().fit(X2,y)
pd.DataFrame(modelo.coef_,columns=X2.columns).T

<a name="roc"></a>
### Curva ROC

In [ ]:
import numpy as np
modelo = LogisticRegression()

# 1 dividir columnas
yColumn = "label"
XColumns = [c for c in df2.columns if c!=yColumn]
X = df[XColumns]
y = df[yColumn]

# 2 preparar train y test
test = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test)

y_score = modelo.fit(X_train, y_train).decision_function(X_test)

from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
v = np.argmax(tpr - fpr)
plt.scatter([fpr[v]],[tpr[v]])
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC, ejemplo autobuses')
plt.legend(loc="lower right")

plt.show()
